In [1]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import time

In [ ]:
! pip install urlextract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from urlextract import URLExtract as extract
extractor = extract()

In [ ]:
response = requests.get('https://www.uvek-gis.admin.ch/BFE/ogd/staging/')
response.status_code

200

In [9]:
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup
def read_url(url):
  
  arr = []
  url = url.replace(" ","%20")
  req = Request(url)
  a = urlopen(req).read()
  soup = BeautifulSoup(a, 'html.parser')
  x = (soup.find_all('a'))
  for i in x:
    file_name = i.extract().get_text()
    url_new = url + file_name
    url_new = url_new.replace(" ","%20")
        
    if(file_name[-1]=='/' and file_name[0]!='.'):
      read_url(url_new)
        
        
    url_new = url_new.replace('https://www.uvek-gis.admin.ch/BFE/ogd/staging/',"");
    if '.csv' in url_new:
      if 'ogd' in url_new:
        arr.append(url_new)

  return arr


url = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'
arr = read_url(url)
arr

[]

In [11]:
arr = ['ogd1_ldp.csv', 'ogd23_ABC.csv', 'ogd623_isfd.csv', 'ogd4_3545.csv']
arr

['ogd1_ldp.csv', 'ogd23_ABC.csv', 'ogd623_isfd.csv', 'ogd4_3545.csv']

In [15]:
info = []
for fileName in arr:
  ogd_id = fileName.partition('_')[0]
  id = ogd_id.replace('ogd', '')

  info.append([fileName, id])

In [16]:
info

[['ogd1_ldp.csv', '1'],
 ['ogd23_ABC.csv', '23'],
 ['ogd623_isfd.csv', '623'],
 ['ogd4_3545.csv', '4']]

In [19]:
def extract_id(fileName):
  ogd_id = fileName.partition('_')[0]

  return ogd_id.replace('ogd', '')

In [20]:
extract_id('ogd623_isfd.csv')

'623'

In [18]:
df = pd.DataFrame(info)
df

,0,1
0,ogd1_ldp.csv,1
1,ogd23_ABC.csv,23
2,ogd623_isfd.csv,623
3,ogd4_3545.csv,4


In [ ]:
import requests

url = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.48',
    #'From': 'jana.stojanovic'  # This is another valid field
}

response = requests.get(url, headers=headers)
response

<Response [403]>

In [ ]:

import requests

HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

r = requests.get(url, headers=HEADERS)
print(r.text)

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta http-equiv="x-ua-compatible" content="ie=edge">
  <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
  <title>403 Forbidden</title>
  <link rel="stylesheet" href="/error_docs/styles.css">
</head>
<body>
<div class="page">
  <div class="main">
    <h1>Server Error</h1>
    <div class="error-code">403</div>
    <h2>Forbidden</h2>
    <p class="lead">You do not have permission to access this document.</p>
    <hr/>
    <p>That's what you can do</p>
    <div class="help-actions">
      <a href="javascript:location.reload();">Reload Page</a>
      <a href="javascript:history.back();">Back to Previous Page</a>
      <a href="/">Home Page</a>
    </div>
  </div>
</div>
</body>
</html>


In [ ]:
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup
def read_url(url):
    url = url.replace(" ","%20")
    req = Request(url)
    a = urlopen(req).read()
    soup = BeautifulSoup(a, 'html.parser')
    x = (soup.find_all('a'))
    for i in x:
        file_name = i.extract().get_text()
        url_new = url + file_name
        url_new = url_new.replace(" ","%20")
        if(file_name[-1]=='/' and file_name[0]!='.'):
            read_url(url_new)
        
        
        url_new = url_new.replace('https://www.uvek-gis.admin.ch/BFE/ogd/staging/',"");
        if '.' in url_new:
          print(url_new)



url = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'
read_url(url)

data-cantons_LNF.csv
data-ch_LNF.csv
data-models-bev_LNF.csv
datapackage.json


check if ending

In [ ]:
import requests
import random

user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
]

r = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
print(r.text)

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta http-equiv="x-ua-compatible" content="ie=edge">
  <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
  <title>403 Forbidden</title>
  <link rel="stylesheet" href="/error_docs/styles.css">
</head>
<body>
<div class="page">
  <div class="main">
    <h1>Server Error</h1>
    <div class="error-code">403</div>
    <h2>Forbidden</h2>
    <p class="lead">You do not have permission to access this document.</p>
    <hr/>
    <p>That's what you can do</p>
    <div class="help-actions">
      <a href="javascript:location.reload();">Reload Page</a>
      <a href="javascript:history.back();">Back to Previous Page</a>
      <a href="/">Home Page</a>
    </div>
  </div>
</div>
</body>
</html>


In [ ]:
import requests

HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        #"Accept-Language": "en-US,en;q=0.5",
        #"Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }

r = requests.patch(url, headers=HEADERS)
print(r.text)

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta http-equiv="x-ua-compatible" content="ie=edge">
  <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">
  <title>405 Method Not Allowed</title>
  <link rel="stylesheet" href="/error_docs/styles.css">
</head>
<body>
<div class="page">
  <div class="main">
    <h1>Server Error</h1>
    <div class="error-code">405</div>
    <h2>Method Not Allowed</h2>
    <p class="lead">The HTTP verb used to access this page is not allowed.</p>
    <hr/>
    <p>That's what you can do</p>
    <div class="help-actions">
      <a href="javascript:location.reload();">Reload Page</a>
      <a href="javascript:history.back();">Back to Previous Page</a>
      <a href="/">Home Page</a>
    </div>
  </div>
</div>
</body>
</html>




In [ ]:
r

<Response [405]>

In [ ]:
url = new URL(urlToRead);
conn = (HttpURLConnection) url.openConnection();
        conn.setRequestMethod("GET");
        is = conn.getInputStream();

In [ ]:
from os import listdir
from os.path import isfile, join
mypath = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

FileNotFoundError: ignored

In [ ]:
data-models-bev_LNF.csv

In [ ]:
import os, glob
path = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'
for filename in glob.glob(os.path.join(path, '*.csv')):
   with open(os.path.join(os.getcwd(), filename), 'r') as f: # open in readonly mode
    print(filename)

In [ ]:
import os
filenames = os.listdir('https://www.uvek-gis.admin.ch/BFE/ogd/staging/')
logFile = open('filenames.txt')
for name in filenames:
    logFile.write(filename)
logFile.close()

FileNotFoundError: ignored

In [ ]:
for  name in os.listdir('https://www.uvek-gis.admin.ch/BFE/ogd/staging/'):
    if os.path.isfile(name): 
      print('file: ', name)
    elif os.path.isdir(name): 
      print( 'dir: ', name)
    elif os.path.islink(name): 
      print( 'link: ', name)
    else: 
      print( 'unknown', name)

FileNotFoundError: ignored

In [ ]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'
#ext = 'iso'

def listFD(url, ext=''):
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')
    return [url + '/' + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]

for file in listFD(url, ext):
    print(file)

In [ ]:
import pathlib

# folder path
dir_path = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'

# to store file names
res = []

# construct path object
d = pathlib.Path(dir_path)

# iterate directory
for entry in d.iterdir():
    # check if it a file
    if entry.is_file():
        res.append(entry)
print(res)

FileNotFoundError: ignored

In [ ]:
from urllib.request import urlopen
from urllib.error import *
 
# try block to read URL

def checkLink(link):

  try:
      html = urlopen(link)
      print(html)
      
  # except block to catch
  # exception
  # and identify error
  except HTTPError as e:
      print("HTTP error", e)
      print(link)
      print()
      return False
      
  except URLError as e:
      print("Opps ! Page not found!", e)
      print(link)
      print()
      return False
  
  else:
      return True

In [ ]:
import urllib.request

with urllib.request.urlopen("https://www.uvek-gis.admin.ch/BFE/ogd/staging/") as url:
    s = url.read()
    # I'm guessing this would output the html source code ?
    print(s)

HTTPError: ignored

In [ ]:
import urllib

link = "https://www.uvek-gis.admin.ch/BFE/ogd/staging/"
f = urllib.urlopen(link)
myfile = f.read()
print(myfile)

AttributeError: ignored

In [ ]:
checkLink('https://www.uvek-gis.admin.ch/BFE/ogd/staging/')

HTTP error HTTP Error 403: Forbidden
https://www.uvek-gis.admin.ch/BFE/ogd/staging/



False

In [ ]:
import glob

# search all files inside a specific folder
# *.* means file name with any extension
dir_path = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'
res = glob.glob(dir_path)
print(res)

[]


In [ ]:
page = requests.get(url).text
page

'<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta charset="utf-8">\n  <meta http-equiv="x-ua-compatible" content="ie=edge">\n  <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n  <title>403 Forbidden</title>\n  <link rel="stylesheet" href="/error_docs/styles.css">\n</head>\n<body>\n<div class="page">\n  <div class="main">\n    <h1>Server Error</h1>\n    <div class="error-code">403</div>\n    <h2>Forbidden</h2>\n    <p class="lead">You do not have permission to access this document.</p>\n    <hr/>\n    <p>That\'s what you can do</p>\n    <div class="help-actions">\n      <a href="javascript:location.reload();">Reload Page</a>\n      <a href="javascript:history.back();">Back to Previous Page</a>\n      <a href="/">Home Page</a>\n    </div>\n  </div>\n</div>\n</body>\n</html>'

In [ ]:
page = requests.get('https://www.uvek-gis.admin.ch/BFE/ogd/staging/data-models-bev_LNF.csv').text
page

'year,quarter,month,rank0model,rank0count,rank1model,rank1count,rank2model,rank2count,rank3model,rank3count,rank4model,rank4count,rank5model,rank5count,rank6model,rank6count,rank7model,rank7count,rank8model,rank8count,rank9model,rank9count,rank10model,rank10count\r\n2020,1,,NISSAN E-NV200,42,RENAULT KANGOO,37,GOUPIL G4,9,MAN EVITO,5,MERCEDES ETGE,5,STREETSCOOTER WORK,4,MAXUS EV80,3,CITROEN BERLINGO,2,PEUGEOT PARTNER,1,RENAULT MASTER,1,ANDERE,3\r\n2020,2,,RENAULT KANGOO,66,NISSAN E-NV200,45,MAXUS EV80,8,MERCEDES EVITO,4,GOUPIL G4,3,OPEL BERLINGO,2,GOUPIL G4L,2,RENAULT MASTER,2,ALKE ATX,2, ,0,ANDERE,3\r\n2020,3,,RENAULT KANGOO,38,NISSAN E-NV200,37,RENAULT MASTER,11,GOUPIL G4,10,MERCEDES EVITO,7,MAXUS EV80,6,MERCEDES ESPRINTER,4,ALKE ATX,3,VW E-CRAFTER,2,GOUPIL G5E,2,ANDERE,0\r\n2020,4,,RENAULT KANGOO,170,TOYOTA PROACE,57,NISSAN E-NV200,42,OPEL VIVARO-E,25,MAXUS EV30,16,RENAULT MASTER,13,PEUGEOT E-EXPERT,11,MAXUS G4,9,MERCEDES ESPRINTER,8,CITROEN E-JUMPY,7,ANDERE,34\r\n2020,5,,RENAULT KAN